# Refactor the Taxi Fare Prediction Problem with a Pipeline

We will refactor the model you built for the Taxi Fare Prediction Problem using:
- Custom encoders for the distance and time features
- OneHot Encoder in order to encode the hour and day of week features
- SimpleImputer to fill missing values
- A simple linear regression
- A pipeline to put all together

Then: 
- train this pipeline
- apply the pipeline on test data
- generate predictions and submit these new predictions to Kaggle

In [2]:
import pandas as pd

## First pipeline

In [4]:
# import the train dataset ( 1000 first rows)
df = pd.read_csv('data/train.csv',nrows=1_000)
df.head()

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2009-06-15 17:26:21.0000001,4.5,2009-06-15 17:26:21 UTC,-73.844311,40.721319,-73.841610,40.712278,1
1,2010-01-05 16:52:16.0000002,16.9,2010-01-05 16:52:16 UTC,-74.016048,40.711303,-73.979268,40.782004,1
2,2011-08-18 00:35:00.00000049,5.7,2011-08-18 00:35:00 UTC,-73.982738,40.761270,-73.991242,40.750562,2
3,2012-04-21 04:30:42.0000001,7.7,2012-04-21 04:30:42 UTC,-73.987130,40.733143,-73.991567,40.758092,1
4,2010-03-09 07:51:00.000000135,5.3,2010-03-09 07:51:00 UTC,-73.968095,40.768008,-73.956655,40.783762,1


In [7]:
# prepare X and y
X = df[['pickup_datetime', 'pickup_longitude',
       'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude',
       'passenger_count']]
y= df["fare_amount"]

X.head()

,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2009-06-15 17:26:21 UTC,-73.844311,40.721319,-73.841610,40.712278,1
1,2010-01-05 16:52:16 UTC,-74.016048,40.711303,-73.979268,40.782004,1
2,2011-08-18 00:35:00 UTC,-73.982738,40.761270,-73.991242,40.750562,2
3,2012-04-21 04:30:42 UTC,-73.987130,40.733143,-73.991567,40.758092,1
4,2010-03-09 07:51:00 UTC,-73.968095,40.768008,-73.956655,40.783762,1


In [8]:
# Hold out ( train and test dplit )
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [9]:
X_train.head()

,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
703,2012-05-21 11:37:42 UTC,-73.992170,40.764320,-73.966214,40.762562,1
311,2009-03-13 17:43:18 UTC,-73.977984,40.754546,-73.958947,40.779093,4
722,2010-10-31 01:12:41 UTC,-74.005771,40.736004,-73.989396,40.718013,1
629,2014-08-05 16:40:00 UTC,-73.954767,40.783890,-73.958958,40.780905,1
0,2009-06-15 17:26:21 UTC,-73.844311,40.721319,-73.841610,40.712278,1


### Custom transformers

With the Taxi Fare Prediction Challenge data, using `BaseEstimator` and `TransformerMixin`, implement:

- a transformer that computes the haversine distance between the pickup and dropoff locations
- a custom encoder that extracts the time features from `pickup_datetime`

In [10]:
import numpy as np

def haversine_vectorized(df, 
                         start_lat="pickup_latitude",
                         start_lon="pickup_longitude",
                         end_lat="dropoff_latitude",
                         end_lon="dropoff_longitude"):
    """ 
        Calculates the great circle distance between two points 
        on the earth (specified in decimal degrees).
        Vectorized version of the haversine distance for pandas df.
        Computes the distance in kms.
    """

    lat_1_rad, lon_1_rad = np.radians(df[start_lat].astype(float)), np.radians(df[start_lon].astype(float))
    lat_2_rad, lon_2_rad = np.radians(df[end_lat].astype(float)), np.radians(df[end_lon].astype(float))
    dlon = lon_2_rad - lon_1_rad
    dlat = lat_2_rad - lat_1_rad

    a = np.sin(dlat / 2.0) ** 2 + np.cos(lat_1_rad) * np.cos(lat_2_rad) * np.sin(dlon / 2.0) ** 2
    c = 2 * np.arcsin(np.sqrt(a))
    return 6371 * c

In [24]:
# create a DistanceTransformer
from sklearn.base import BaseEstimator, TransformerMixin

class DistanceTransformer(BaseEstimator, TransformerMixin):
    """
        Computes the haversine distance between two GPS points.
        Returns a copy of the DataFrame X with only one column: 'distance'.
    """

    def __init__(self,
                 start_lat="pickup_latitude",
                 start_lon="pickup_longitude",
                 end_lat="dropoff_latitude",
                 end_lon="dropoff_longitude"):
        # A COMPPLETER
        self.start_lat=start_lat
        self.start_lon=start_lon
        self.end_lat=end_lat
        self.end_lon=end_lon
        
    def fit(self, X, y=None):
        # A COMPLETER
        return self

    def transform(self, X, y=None):
        X_= X.copy()
        
        X_['distance'] = haversine_vectorized(df, 
                                         self.start_lat,
                                         self.start_lon,
                                         self.end_lat,
                                         self.end_lon)
        return X_[['distance']]

In [27]:
# test the DistanceTransformer

dist_trans = DistanceTransformer()
distance = dist_trans.fit_transform(X_train, y_train)
distance.head()

,distance
703,2.194746
311,3.165517
722,2.430238
629,0.484442
0,1.030764


In [63]:
# create a TimeFeaturesEncoder
class TimeFeaturesEncoder(BaseEstimator, TransformerMixin):
    """
        Extracts the day of week (dow), the hour, the month and the year from a time column.
        Returns a copy of the DataFrame X with only four columns: 'dow', 'hour', 'month', 'year'.
    """

    def __init__(self, time_column):
       # A COMPLETER
        self.time_column = time_column
    

    def fit(self, X, y=None):
        # A COMPLETER
        return self

    def transform(self, X, y=None):
        # A COMPLETER 
        X_=X.copy()
        X_[self.time_column]= pd.to_datetime(X_[self.time_column], format='%Y-%m-%d %H:%M:%S UTC')

             # extract hour, day of week, month, and year
        X_['hour'] = X_[self.time_column].dt.hour
        X_['dow'] = X_[self.time_column].dt.dayofweek
        X_['month'] = X_[self.time_column].dt.month
        X_['year'] = X_[self.time_column].dt.year
        return X_[['dow', 'hour', 'month', 'year']]

In [64]:
# test the TimeFeaturesEncoder

time_enc = TimeFeaturesEncoder('pickup_datetime')
time_features = time_enc.fit_transform(X_train, y_train)
time_features.head()

,dow,hour,month,year
703,0,11,5,2012
311,4,17,3,2009
722,6,1,10,2010
629,1,16,8,2014
0,0,17,6,2009


###  Prepocessing pipeline

In [65]:
# visualizing pipelines in HTML
from sklearn import set_config; set_config(display='diagram')

#### Distance pipeline

Create a pipeline for distances:
- convert the pickup and dropoff coordinates into distances with the DistanceTransformer
- standardize these distances

In [66]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler


# create distance pipeline dist_pipe
dist_pip = Pipeline([('dist_trans', DistanceTransformer()),
                ('stdscaler', StandardScaler())])

# display distance pipeline
dist_pip


Pipeline(steps=[('dist_trans', DistanceTransformer()),
                ('stdscaler', StandardScaler())])

#### Time features pipeline

Create a pipeline for time features
- extract time features from pickup datetime with the TimeFeaturesEncoder
- encode these categorical time features with the OneHotEncoder

In [68]:
from sklearn.preprocessing import OneHotEncoder
# create time pipeline time_pipe
time_pipe = Pipeline([('time_enc',
                 TimeFeaturesEncoder(time_column='pickup_datetime')),
                ('ohe', OneHotEncoder(handle_unknown='ignore'))])

# display time pipeline
time_pipe

Pipeline(steps=[('time_enc',
                 TimeFeaturesEncoder(time_column='pickup_datetime')),
                ('ohe', OneHotEncoder(handle_unknown='ignore'))])

#### Preprocessing pipeline

Wrap up the distance pipeline and the time pipeline into a preprocessing pipeline.

In [71]:
import time
from sklearn.compose import ColumnTransformer

# create preprocessing pipeline preproc_pipe
preproc_pipe = ColumnTransformer([
    ('time', time_pipe, ['pickup_datetime']),
    ('distance', dist_pip, ['pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude'])
])
# display preprocessing pipeline
preproc_pipe

ColumnTransformer(transformers=[('time',
                                 Pipeline(steps=[('time_enc',
                                                  TimeFeaturesEncoder(time_column='pickup_datetime')),
                                                 ('ohe',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 ['pickup_datetime']),
                                ('distance',
                                 Pipeline(steps=[('dist_trans',
                                                  DistanceTransformer()),
                                                 ('stdscaler',
                                                  StandardScaler())]),
                                 ['pickup_longitude', 'pickup_latitude',
                                  'dropoff_longitude', 'dropoff_latitude'])])

### Model pipeline

Create a pipeline containing the preprocessing and the regression model of your choice.

In [16]:
from sklearn.linear_model import LinearRegression

# Add the model of your choice to the pipeline nammed pipe

# display the pipeline with model


Pipeline(steps=[('preproc',
                 ColumnTransformer(transformers=[('distance',
                                                  Pipeline(steps=[('dist_trans',
                                                                   DistanceTransformer()),
                                                                  ('stdscaler',
                                                                   StandardScaler())]),
                                                  ['pickup_latitude',
                                                   'pickup_longitude',
                                                   'dropoff_latitude',
                                                   'dropoff_longitude']),
                                                 ('time',
                                                  Pipeline(steps=[('time_enc',
                                                                   TimeFeaturesEncoder(time_column='pickup_datetime')),
                                                                  ('ohe',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['pickup_datetime'])])),
                ('linear_model', LinearRegression())])

### Training and performance

Train the pipelined model and compute the prediction on the test set:

In [17]:
# train the pipelined model


# compute y_pred on the test set


Use the RMSE to evaluate the performance of the model:

In [18]:
def compute_rmse(y_pred, y_true):
    
    return # A COMPLETER

In [19]:
# call compute_rmse
compute_rmse(y_pred, y_test)

10.175329127383847

## Complete the workflow with a pipeline

Here we will implement the whole workflow for our Taxifare kaggle challenge.

For that we will refactor the code in functions for more clarity.

Implement the following functions:
- `get_data()` to fetch the data 
- `clean_data()` to clean the data
- `get_pipeline()` to get the pipeline defined earlier
- `train()` to train our model
- `evaluate()` to evaluate our model on test data

In [25]:
# implement get_data() function
def get_data(nrows=10000):
    '''returns a DataFrame with nrows from s3 bucket'''
    # A COMPLETER
    return df

In [26]:
# implement clean_data() function
def clean_data(df, test=False):
    '''returns a DataFrame without outliers and missing values'''
    # A COMPLETER
    return df

In [27]:
# implement set_pipeline() function
def set_pipeline():
    '''returns a pipelined model'''
     # A  COMPLETER
    return pipe

In [28]:
# implement train() function
def train(X_train, y_train, pipeline):
    '''returns a trained pipelined model'''
   # A COMPLETER
    return pipeline

In [29]:
# implement evaluate() function
def evaluate(X_test, y_test, pipeline):
    '''returns the value of the RMSE'''
    # A COMPLETER
    print(rmse)
    return rmse

### Test the complete worflow

Use the above functions to test the complete workflow.

In [30]:
# store the data in a DataFrame
df = get_data()

# set X and y
y = df["fare_amount"]
X = df.drop("fare_amount", axis=1)

# hold out
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.15)

# build pipeline
pipeline = set_pipeline()

# train the pipeline
train(X_train, y_train, pipeline)

# evaluate the pipeline
rmse = evaluate(X_val, y_val, pipeline)

9.396264153241109


### Congrats!

Now we are ready to convert this complete workflow into a packaged code 🚀